---

## 1. Justificativa: Alternativa ao Discord

### Por que não usar Discord?

A integração direta com Discord apresenta diversos desafios técnicos que a tornam **inviável** para execução em ambiente de notebook:

#### 1.1 Complexidade de Autenticação OAuth
- Discord exige criação de aplicação no [Discord Developer Portal](https://discord.com/developers/applications)
- Necessita configuração de Bot Token e permissões específicas
- Requer Gateway Intents (Privileged Intents) que precisam ser aprovados manualmente
- Tokens sensíveis não devem ser expostos em notebooks compartilháveis

#### 1.2 Requisitos de Infraestrutura Persistente
- Discord bots precisam de **conexão WebSocket persistente** com os servidores
- Notebook executa células de forma **não-persistente** (execução pontual)
- Desconexões frequentes invalidariam o streaming contínuo
- Rate limits do Discord (50 requests/segundo) complicam testes

#### 1.3 Dependências de Servidor/Canal
- Necessita servidor Discord específico com permissões de administrador
- Configuração de canais e webhooks externos ao ambiente de teste
- Não é reproduzível sem acesso ao servidor Discord configurado

### Alternativa Escolhida: Simulador de Streaming HTTP/RSS

Para garantir **reprodutibilidade** e **execução autocontida**, implementamos:

1. **Producer Python** que simula mensagens de rede social
2. **Fonte de dados**: API pública de notícias (NewsAPI) ou geração sintética
3. **Publicação em Kafka** no tópico `social-input`
4. **Formato JSON** compatível com pipeline Spark

**Vantagens:**
- ✅ Totalmente executável dentro do notebook
- ✅ Sem dependências de contas externas
- ✅ Reproduzível em qualquer ambiente
- ✅ Controle total sobre taxa de mensagens
- ✅ Simula características de rede social (timestamp, autor, texto)

**Referências:**
- [Discord Developer Docs - Rate Limits](https://discord.com/developers/docs/topics/rate-limits)
- [Kafka Documentation - Use Cases](https://kafka.apache.org/documentation/#uses)

---

## 2. Configuração do Ambiente

### 2.1 Importações e Configurações Iniciais

In [ ]:
import os
import sys
import time
import json
import subprocess
import threading
from datetime import datetime
from pathlib import Path

# Configurações globais
KAFKA_BOOTSTRAP_SERVERS = "localhost:29092"
KAFKA_TOPIC_INPUT = "social-input"
KAFKA_TOPIC_OUTPUT = "wordcount-output"
ELASTICSEARCH_HOST = "http://localhost:9200"
ELASTICSEARCH_INDEX = "wordcount-realtime"
SPARK_MASTER = "spark://localhost:7077"

# Diretório do projeto
PROJECT_DIR = Path("/home/edilberto/pspd/atividade-extraclasse-2-pspd/spark")
os.chdir(PROJECT_DIR)

print("✓ Configurações carregadas")
print(f"  - Kafka: {KAFKA_BOOTSTRAP_SERVERS}")
print(f"  - Tópico entrada: {KAFKA_TOPIC_INPUT}")
print(f"  - Tópico saída: {KAFKA_TOPIC_OUTPUT}")
print(f"  - Elasticsearch: {ELASTICSEARCH_HOST}")
print(f"  - Índice ES: {ELASTICSEARCH_INDEX}")

### 2.2 Iniciar Infraestrutura Docker

In [ ]:
%%bash
cd /home/edilberto/pspd/atividade-extraclasse-2-pspd/spark

echo "=== Iniciando containers Docker ==="
docker-compose up -d

echo ""
echo "=== Aguardando inicialização dos serviços (30s) ==="
sleep 30

echo ""
echo "=== Status dos containers ==="
docker-compose ps

### 2.3 Verificar Saúde dos Serviços

In [ ]:
import requests

def check_service(name, url, timeout=5):
    """Verifica se um serviço está respondendo"""
    try:
        response = requests.get(url, timeout=timeout)
        if response.status_code < 400:
            print(f"✓ {name}: OK (status {response.status_code})")
            return True
        else:
            print(f"✗ {name}: ERRO (status {response.status_code})")
            return False
    except Exception as e:
        print(f"✗ {name}: INACESSÍVEL ({str(e)})")
        return False

print("=== Verificando Serviços ===")
es_ok = check_service("Elasticsearch", "http://localhost:9200")
kibana_ok = check_service("Kibana", "http://localhost:5601")
spark_ok = check_service("Spark Master", "http://localhost:8080")

if es_ok and kibana_ok and spark_ok:
    print("\n✓ Todos os serviços estão operacionais!")
else:
    print("\n⚠ Alguns serviços não responderam. Aguarde mais tempo ou verifique logs.")

### 2.4 Criar Tópicos Kafka

In [ ]:
%%bash
echo "=== Criando tópicos Kafka ==="

# Criar tópico de entrada
docker exec kafka kafka-topics --create \
  --bootstrap-server localhost:9092 \
  --topic social-input \
  --partitions 3 \
  --replication-factor 1 \
  --if-not-exists

# Criar tópico de saída
docker exec kafka kafka-topics --create \
  --bootstrap-server localhost:9092 \
  --topic wordcount-output \
  --partitions 3 \
  --replication-factor 1 \
  --if-not-exists

echo ""
echo "=== Listando tópicos ==="
docker exec kafka kafka-topics --list --bootstrap-server localhost:9092

---

## 3. Producer Kafka - Simulador de Rede Social

### 3.1 Instalação de Dependências Python

In [ ]:
!pip install -q kafka-python elasticsearch pyspark==3.5.0

### 3.2 Implementação do Producer

In [ ]:
from kafka import KafkaProducer
import random
import time
import json
from datetime import datetime

# Dataset sintético simulando mensagens de rede social
SAMPLE_MESSAGES = [
    "Apache Spark is amazing for big data processing and analytics",
    "Learning distributed systems with Kafka and Spark",
    "Real-time data processing using Structured Streaming",
    "Elasticsearch and Kibana provide powerful visualization tools",
    "Docker containers make deployment much easier",
    "Big data analytics requires scalable infrastructure",
    "Cloud computing enables elastic scalability",
    "Machine learning models benefit from distributed training",
    "Parallel computing accelerates data processing pipelines",
    "Kafka provides reliable message streaming capabilities",
    "Python is a versatile language for data science",
    "Distributed databases handle massive datasets efficiently",
    "Stream processing enables real-time analytics",
    "Microservices architecture improves system modularity",
    "Data pipelines transform raw data into insights"
]

SAMPLE_USERS = ["alice", "bob", "charlie", "diana", "eve", "frank", "grace", "henry"]

class SocialMediaProducer:
    """Producer que simula mensagens de rede social"""
    
    def __init__(self, bootstrap_servers, topic):
        self.producer = KafkaProducer(
            bootstrap_servers=bootstrap_servers,
            value_serializer=lambda v: json.dumps(v).encode('utf-8')
        )
        self.topic = topic
        self.running = False
        self.message_count = 0
        
    def generate_message(self):
        """Gera uma mensagem sintética"""
        return {
            "user": random.choice(SAMPLE_USERS),
            "text": random.choice(SAMPLE_MESSAGES),
            "timestamp": datetime.now().isoformat(),
            "platform": "SimulatedSocial",
            "message_id": self.message_count
        }
    
    def start(self, duration_seconds=300, messages_per_second=2):
        """Inicia o producer por um período determinado"""
        self.running = True
        start_time = time.time()
        interval = 1.0 / messages_per_second
        
        print(f"▶ Producer iniciado: {messages_per_second} msgs/seg por {duration_seconds}s")
        
        try:
            while self.running and (time.time() - start_time) < duration_seconds:
                message = self.generate_message()
                self.producer.send(self.topic, value=message)
                self.message_count += 1
                
                if self.message_count % 10 == 0:
                    print(f"  [{self.message_count}] Enviado: {message['user']}: {message['text'][:50]}...")
                
                time.sleep(interval)
        
        except KeyboardInterrupt:
            print("\n⏸ Producer interrompido pelo usuário")
        finally:
            self.stop()
    
    def stop(self):
        """Para o producer"""
        self.running = False
        self.producer.flush()
        self.producer.close()
        print(f"■ Producer parado. Total de mensagens: {self.message_count}")

# Criar producer (será iniciado posteriormente)
producer = SocialMediaProducer(KAFKA_BOOTSTRAP_SERVERS, KAFKA_TOPIC_INPUT)
print("✓ Producer configurado e pronto")

### 3.3 Testar Producer (Envio de Amostra)

In [ ]:
# Enviar 20 mensagens de teste
print("=== Teste do Producer (20 mensagens) ===")
test_producer = SocialMediaProducer(KAFKA_BOOTSTRAP_SERVERS, KAFKA_TOPIC_INPUT)
test_producer.start(duration_seconds=10, messages_per_second=2)

### 3.4 Verificar Mensagens no Tópico Kafka

In [ ]:
%%bash
echo "=== Últimas 5 mensagens no tópico social-input ==="
docker exec kafka kafka-console-consumer \
  --bootstrap-server localhost:9092 \
  --topic social-input \
  --from-beginning \
  --max-messages 5 \
  --timeout-ms 5000 2>/dev/null | tail -5

---

## 4. Pipeline Spark Structured Streaming

### 4.1 Configuração da Sessão Spark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, explode, split, window, from_json, to_json, struct, current_timestamp
)
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType

# Criar sessão Spark com suporte a Kafka
spark = SparkSession.builder \
    .appName("B2_SocialMedia_WordCount") \
    .master("local[*]") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0") \
    .config("spark.sql.streaming.forceDeleteTempCheckpointLocation", "true") \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")

print("✓ Sessão Spark criada")
print(f"  - Versão: {spark.version}")
print(f"  - App: {spark.sparkContext.appName}")

### 4.2 Schema das Mensagens de Entrada

In [ ]:
# Schema JSON das mensagens do producer
message_schema = StructType([
    StructField("user", StringType(), True),
    StructField("text", StringType(), True),
    StructField("timestamp", StringType(), True),
    StructField("platform", StringType(), True),
    StructField("message_id", IntegerType(), True)
])

print("✓ Schema definido:")
message_schema.printTreeString()

### 4.3 Ler Stream do Kafka (Entrada)

In [ ]:
# Ler mensagens do tópico de entrada
raw_stream = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", KAFKA_BOOTSTRAP_SERVERS) \
    .option("subscribe", KAFKA_TOPIC_INPUT) \
    .option("startingOffsets", "latest") \
    .load()

# Parse JSON e extrair campos
messages = raw_stream.selectExpr("CAST(value AS STRING) as json") \
    .select(from_json(col("json"), message_schema).alias("data")) \
    .select("data.*") \
    .withColumn("event_time", current_timestamp())

print("✓ Stream de entrada configurado")
print("  Schema do stream:")
messages.printSchema()

### 4.4 Processamento: Word Count com Janelas Temporais

In [ ]:
# Extrair palavras do texto
words = messages.select(
    explode(split(col("text"), "\\s+")).alias("word"),
    col("event_time"),
    col("user")
).filter(col("word") != "")

# Normalizar palavras (lowercase, remover pontuação)
from pyspark.sql.functions import lower, regexp_replace

words_clean = words.withColumn(
    "word", 
    lower(regexp_replace(col("word"), "[^a-zA-Z0-9]", ""))
).filter(col("word") != "")

# Agregação com janelas de 30 segundos
WINDOW_DURATION = "30 seconds"
SLIDE_DURATION = "10 seconds"

word_counts = words_clean \
    .withWatermark("event_time", "1 minute") \
    .groupBy(
        window(col("event_time"), WINDOW_DURATION, SLIDE_DURATION),
        col("word")
    ) \
    .count() \
    .select(
        col("window.start").alias("window_start"),
        col("window.end").alias("window_end"),
        col("word"),
        col("count")
    )

print("✓ Pipeline de word count configurado")
print(f"  - Janela: {WINDOW_DURATION}")
print(f"  - Slide: {SLIDE_DURATION}")
print(f"  - Watermark: 1 minuto")

### 4.5 Escrever Resultados no Kafka (Saída)

In [ ]:
# Converter para JSON
output_json = word_counts.select(
    to_json(struct(
        col("word"),
        col("count"),
        col("window_start"),
        col("window_end")
    )).alias("value")
)

# Escrever no tópico de saída
query_kafka = output_json.writeStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", KAFKA_BOOTSTRAP_SERVERS) \
    .option("topic", KAFKA_TOPIC_OUTPUT) \
    .option("checkpointLocation", "/tmp/spark-checkpoint-kafka") \
    .outputMode("update") \
    .start()

print("✓ Stream de saída para Kafka iniciado")
print(f"  - Query ID: {query_kafka.id}")
print(f"  - Tópico: {KAFKA_TOPIC_OUTPUT}")

### 4.6 Visualização em Console (Debug)

In [ ]:
# Query adicional para visualizar resultados no console
query_console = word_counts.writeStream \
    .format("console") \
    .option("truncate", "false") \
    .option("numRows", 20) \
    .outputMode("update") \
    .start()

print("✓ Stream de debug (console) iniciado")
print(f"  - Query ID: {query_console.id}")

---

## 5. Iniciar Producer em Background

### 5.1 Executar Producer em Thread Separada

In [ ]:
import threading

# Configuração do producer
PRODUCER_DURATION = 180  # 3 minutos
MESSAGES_PER_SECOND = 3

# Criar novo producer
background_producer = SocialMediaProducer(KAFKA_BOOTSTRAP_SERVERS, KAFKA_TOPIC_INPUT)

# Função para executar em thread
def run_producer():
    background_producer.start(
        duration_seconds=PRODUCER_DURATION,
        messages_per_second=MESSAGES_PER_SECOND
    )

# Iniciar thread
producer_thread = threading.Thread(target=run_producer, daemon=True)
producer_thread.start()

print(f"✓ Producer iniciado em background")
print(f"  - Duração: {PRODUCER_DURATION}s")
print(f"  - Taxa: {MESSAGES_PER_SECOND} msgs/seg")
print(f"  - Total esperado: ~{PRODUCER_DURATION * MESSAGES_PER_SECOND} mensagens")

### 5.2 Monitorar Queries Spark (Aguardar Processamento)

In [ ]:
import time

print("=== Monitorando Queries Spark ===")
print("Aguarde 60 segundos para acumular dados...\n")

for i in range(6):
    time.sleep(10)
    print(f"[{(i+1)*10}s] Query Kafka: {query_kafka.status['message']}")
    print(f"[{(i+1)*10}s] Query Console: {query_console.status['message']}")
    
    # Mostrar progresso
    if 'numInputRows' in query_kafka.lastProgress:
        input_rows = query_kafka.lastProgress['numInputRows']
        print(f"       → Linhas processadas no último batch: {input_rows}")
    print()

print("✓ Processamento em andamento. Verifique console acima para word counts.")

---

## 6. Consumer Elasticsearch

### 6.1 Criar Índice no Elasticsearch

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk

# Conectar ao Elasticsearch
es = Elasticsearch([ELASTICSEARCH_HOST])

# Verificar conexão
if es.ping():
    print("✓ Conectado ao Elasticsearch")
    print(f"  - Cluster: {es.info()['cluster_name']}")
    print(f"  - Versão: {es.info()['version']['number']}")
else:
    print("✗ Falha ao conectar no Elasticsearch")

# Criar índice com mapping
index_mapping = {
    "mappings": {
        "properties": {
            "word": {"type": "keyword"},
            "count": {"type": "integer"},
            "window_start": {"type": "date"},
            "window_end": {"type": "date"},
            "indexed_at": {"type": "date"}
        }
    }
}

# Deletar índice existente (se houver)
if es.indices.exists(index=ELASTICSEARCH_INDEX):
    es.indices.delete(index=ELASTICSEARCH_INDEX)
    print(f"  - Índice '{ELASTICSEARCH_INDEX}' deletado")

# Criar novo índice
es.indices.create(index=ELASTICSEARCH_INDEX, body=index_mapping)
print(f"✓ Índice '{ELASTICSEARCH_INDEX}' criado com sucesso")

### 6.2 Consumer Kafka → Elasticsearch

In [ ]:
from kafka import KafkaConsumer
import json
from datetime import datetime

class ElasticsearchConsumer:
    """Consumer que indexa word counts no Elasticsearch"""
    
    def __init__(self, bootstrap_servers, topic, es_client, es_index):
        self.consumer = KafkaConsumer(
            topic,
            bootstrap_servers=bootstrap_servers,
            value_deserializer=lambda m: json.loads(m.decode('utf-8')),
            auto_offset_reset='latest',
            enable_auto_commit=True,
            group_id='elasticsearch-consumer-group'
        )
        self.es = es_client
        self.index = es_index
        self.running = False
        self.document_count = 0
        
    def start(self, duration_seconds=120, batch_size=50):
        """Inicia o consumer por um período determinado"""
        self.running = True
        start_time = time.time()
        batch = []
        
        print(f"▶ Consumer Elasticsearch iniciado ({duration_seconds}s)")
        
        try:
            for message in self.consumer:
                if (time.time() - start_time) > duration_seconds:
                    break
                
                data = message.value
                
                # Preparar documento
                doc = {
                    "_index": self.index,
                    "_source": {
                        "word": data.get("word"),
                        "count": data.get("count"),
                        "window_start": data.get("window_start"),
                        "window_end": data.get("window_end"),
                        "indexed_at": datetime.now().isoformat()
                    }
                }
                batch.append(doc)
                self.document_count += 1
                
                # Indexar em lote
                if len(batch) >= batch_size:
                    success, _ = bulk(self.es, batch)
                    print(f"  [{self.document_count}] Indexados {success} documentos")
                    batch = []
            
            # Indexar documentos restantes
            if batch:
                success, _ = bulk(self.es, batch)
                print(f"  [Final] Indexados {success} documentos")
        
        except KeyboardInterrupt:
            print("\n⏸ Consumer interrompido pelo usuário")
        finally:
            self.stop()
    
    def stop(self):
        """Para o consumer"""
        self.running = False
        self.consumer.close()
        print(f"■ Consumer parado. Total de documentos: {self.document_count}")

# Criar consumer
es_consumer = ElasticsearchConsumer(
    KAFKA_BOOTSTRAP_SERVERS,
    KAFKA_TOPIC_OUTPUT,
    es,
    ELASTICSEARCH_INDEX
)

print("✓ Consumer Elasticsearch configurado")

### 6.3 Executar Consumer em Background

In [ ]:
CONSUMER_DURATION = 120  # 2 minutos

def run_consumer():
    es_consumer.start(duration_seconds=CONSUMER_DURATION, batch_size=30)

# Iniciar thread do consumer
consumer_thread = threading.Thread(target=run_consumer, daemon=True)
consumer_thread.start()

print(f"✓ Consumer iniciado em background")
print(f"  - Duração: {CONSUMER_DURATION}s")
print(f"  - Batch size: 30 documentos")

### 6.4 Aguardar e Verificar Indexação

In [ ]:
print("=== Aguardando indexação (90 segundos) ===")
time.sleep(90)

# Verificar contagem de documentos
es.indices.refresh(index=ELASTICSEARCH_INDEX)
count = es.count(index=ELASTICSEARCH_INDEX)['count']

print(f"\n✓ Documentos indexados: {count}")

# Mostrar amostra
if count > 0:
    print("\n=== Amostra de 10 word counts ===")
    result = es.search(
        index=ELASTICSEARCH_INDEX,
        body={
            "size": 10,
            "sort": [{"count": {"order": "desc"}}]
        }
    )
    
    for hit in result['hits']['hits']:
        doc = hit['_source']
        print(f"  {doc['word']:20s} → {doc['count']:3d} ocorrências")
else:
    print("⚠ Nenhum documento indexado ainda. Aguarde mais tempo.")

---

## 7. Visualização no Kibana

### 7.1 Instruções para Criar Nuvem de Palavras

#### Passo 1: Acessar Kibana
1. Abra o navegador em: **http://localhost:5601**
2. Aguarde o Kibana carregar (pode levar alguns segundos)

#### Passo 2: Criar Index Pattern
1. No menu lateral, clique em **"Stack Management"** (ícone de engrenagem)
2. Em **"Kibana"**, clique em **"Data Views"** (ou **"Index Patterns"**)
3. Clique em **"Create data view"**
4. Configure:
   - **Name:** `WordCount Real-Time`
   - **Index pattern:** `wordcount-realtime*`
   - **Timestamp field:** `window_start`
5. Clique em **"Create data view"**

#### Passo 3: Criar Visualização Tag Cloud
1. No menu lateral, clique em **"Visualize Library"** (ícone de gráfico)
2. Clique em **"Create visualization"**
3. Selecione o tipo **"Tag Cloud"** (nuvem de tags)
4. Selecione o data view **"WordCount Real-Time"**
5. Configure a visualização:
   - **Buckets → Tags:**
     - Aggregation: `Terms`
     - Field: `word.keyword`
     - Order By: `Metric: Count`
     - Order: `Descending`
     - Size: `50` (top 50 palavras)
   - **Metrics:**
     - Aggregation: `Sum`
     - Field: `count`
6. Clique em **"Update"** (▶️) para aplicar
7. Ajuste o time range no canto superior direito (ex: "Last 15 minutes")
8. Salve a visualização:
   - Clique em **"Save"** no topo
   - Nome: `Word Cloud - Social Media Stream`

#### Passo 4: Criar Dashboard
1. No menu lateral, clique em **"Dashboard"**
2. Clique em **"Create dashboard"**
3. Clique em **"Add"** → Selecione `Word Cloud - Social Media Stream`
4. Adicione visualizações complementares:
   - **Vertical Bar**: Count por janela temporal
   - **Data Table**: Top 20 palavras
   - **Metric**: Total de palavras únicas
5. Configure auto-refresh:
   - Clique no relógio no topo
   - Selecione **"Auto-refresh"** → `10 seconds`
6. Salve o dashboard:
   - Nome: `B2 - Real-Time Word Count Analytics`

### 7.2 Alternativas ao Tag Cloud

Se o Tag Cloud não estiver disponível na sua versão do Kibana:

**Opção A: Horizontal Bar Chart**
- Tipo: `Horizontal Bar`
- Y-axis: `word.keyword` (Terms, top 30)
- X-axis: `count` (Sum)
- Visualiza as palavras mais frequentes em barras horizontais

**Opção B: Data Table**
- Tipo: `Data Table`
- Rows: `word.keyword` (Terms, top 50)
- Metrics: `count` (Sum)
- Visualiza tabela ordenada por contagem

**Opção C: Treemap**
- Tipo: `Treemap`
- Groups: `word.keyword` (Terms, top 40)
- Size: `count` (Sum)
- Visualiza palavras em blocos proporcionais à frequência

### 7.3 Screenshot do Dashboard

**AÇÃO MANUAL REQUERIDA:**
1. Após criar o dashboard, tire um screenshot completo
2. Salve como: `resultados_spark/kibana_dashboard_wordcloud.png`
3. Capture também a visualização Tag Cloud isolada
4. Salve como: `resultados_spark/kibana_tagcloud_detail.png`

### 7.4 Verificar Dados no Kibana via API

In [ ]:
# Verificar se o índice está visível no Kibana
import requests

kibana_url = "http://localhost:5601"

print("=== Verificação Kibana ===")
try:
    # Check Kibana status
    response = requests.get(f"{kibana_url}/api/status")
    if response.status_code == 200:
        print("✓ Kibana está acessível")
        print(f"  URL: {kibana_url}")
        print(f"\n📊 Acesse o Kibana e siga as instruções acima para criar a visualização")
    else:
        print(f"⚠ Kibana retornou status {response.status_code}")
except Exception as e:
    print(f"✗ Erro ao conectar no Kibana: {e}")

# Mostrar estatísticas do índice
print(f"\n=== Estatísticas do Índice '{ELASTICSEARCH_INDEX}' ===")
stats = es.indices.stats(index=ELASTICSEARCH_INDEX)
print(f"  - Total de documentos: {stats['_all']['total']['docs']['count']}")
print(f"  - Tamanho em disco: {stats['_all']['total']['store']['size_in_bytes'] / 1024:.2f} KB")

---

## 8. Parar Streams e Limpar Recursos

### 8.1 Parar Queries Spark

In [ ]:
print("=== Parando Queries Spark ===")

# Parar query Kafka
if query_kafka.isActive:
    query_kafka.stop()
    print("✓ Query Kafka parada")

# Parar query Console
if query_console.isActive:
    query_console.stop()
    print("✓ Query Console parada")

# Aguardar threads terminarem
time.sleep(5)

print("\n✓ Todas as queries foram paradas")

### 8.2 Fechar Sessão Spark

In [ ]:
spark.stop()
print("✓ Sessão Spark encerrada")

### 8.3 Estatísticas Finais

In [ ]:
print("=== ESTATÍSTICAS FINAIS ===")

# Contagem final no Elasticsearch
es.indices.refresh(index=ELASTICSEARCH_INDEX)
final_count = es.count(index=ELASTICSEARCH_INDEX)['count']

print(f"\n📊 Documentos indexados no Elasticsearch: {final_count}")

# Top 20 palavras
if final_count > 0:
    print("\n📈 Top 20 Palavras Mais Frequentes:")
    result = es.search(
        index=ELASTICSEARCH_INDEX,
        body={
            "size": 0,
            "aggs": {
                "top_words": {
                    "terms": {
                        "field": "word.keyword",
                        "size": 20,
                        "order": {"total_count": "desc"}
                    },
                    "aggs": {
                        "total_count": {
                            "sum": {"field": "count"}
                        }
                    }
                }
            }
        }
    )
    
    for i, bucket in enumerate(result['aggregations']['top_words']['buckets'], 1):
        word = bucket['key']
        count = int(bucket['total_count']['value'])
        print(f"  {i:2d}. {word:20s} → {count:4d} ocorrências")

# Verificar tópicos Kafka
print("\n📨 Tópicos Kafka:")
!docker exec kafka kafka-topics --list --bootstrap-server localhost:9092

print("\n✓ Pipeline B2 executado com sucesso!")

### 8.4.7 Visualização de Sentimentos no Kibana (OPCIONAL)

**Após executar o pipeline de sentimentos**, criar visualizações no Kibana:

#### 1. Criar Data View para Sentimentos
- Stack Management → Data Views
- Name: `Social Sentiment Analysis`
- Index pattern: `social-sentiment*`
- Timestamp: `timestamp`

#### 2. Visualização: Pie Chart - Distribuição de Sentimentos
- Tipo: **Pie Chart**
- Metric: Count
- Buckets: Terms by `sentiment_classification.keyword`
- Mostra proporção de mensagens positivas/neutras/negativas

#### 3. Visualização: Line Chart - Sentimento ao Longo do Tempo
- Tipo: **Line**
- X-axis: Date Histogram on `timestamp`
- Y-axis: Average of `sentiment_compound`
- Split series: Terms by `sentiment_classification.keyword`
- Mostra evolução temporal dos sentimentos

#### 4. Visualização: Data Table - Top Mensagens por Sentimento
- Tipo: **Data Table**
- Rows: Terms by `sentiment_classification.keyword`
- Metrics: 
  - Count
  - Average of `sentiment_compound`
  - Min/Max of `sentiment_compound`

#### 5. Dashboard Completo de Sentimentos
Combinar:
- Pie Chart (distribuição)
- Line Chart (temporal)
- Data Table (estatísticas)
- Metric: Total de mensagens analisadas
- Filter: Por classificação de sentimento

### Referências Adicionais para Análise de Sentimentos

1. **VADER Original:**
   - Hutto & Gilbert (2014) - [Paper ICWSM](http://comp.social.gatech.edu/papers/icwsm14.vader.hutto.pdf)

2. **Implementação Python:**
   - [vaderSentiment GitHub](https://github.com/cjhutto/vaderSentiment)
   - [Documentation](https://github.com/cjhutto/vaderSentiment#about-the-scoring)

3. **Alternativas:**
   - TextBlob: Mais simples, menos preciso para redes sociais
   - BERT/Transformers: Mais preciso, requer GPU e treinamento
   - NLTK: Requer corpus e mais configuração

4. **Aplicações em Streaming:**
   - Liu, B. (2015). Sentiment Analysis: Mining Opinions, Sentiments, and Emotions. Cambridge University Press.
   - Medhat, W., Hassan, A., & Korashy, H. (2014). Sentiment analysis algorithms and applications: A survey. Ain Shams Engineering Journal.

**Diferencial deste Trabalho:**
- Integração nativa com Kafka Streaming (não batch)
- Indexação em tempo real no Elasticsearch
- Visualização dinâmica no Kibana
- Pipeline 100% em notebook (reproduzível)

In [ ]:
# OPCIONAL: Executar consumer com análise de sentimentos
# Descomentar para executar

# sentiment_consumer = SentimentElasticsearchConsumer(
#     KAFKA_BOOTSTRAP_SERVERS,
#     "sentiment-input",
#     es,
#     SENTIMENT_INDEX
# )

# def run_sentiment_consumer():
#     sentiment_consumer.start(duration_seconds=120, batch_size=20)

# consumer_sentiment_thread = threading.Thread(target=run_sentiment_consumer, daemon=True)
# consumer_sentiment_thread.start()

# print("✓ Consumer de sentimentos iniciado (120s, batch=20)")

In [ ]:
# OPCIONAL: Executar producer com sentimentos
# Descomentar para executar

# sentiment_producer = SentimentProducer(KAFKA_BOOTSTRAP_SERVERS, "sentiment-input")

# def run_sentiment_producer():
#     sentiment_producer.start(duration_seconds=120, messages_per_second=2)

# sentiment_thread = threading.Thread(target=run_sentiment_producer, daemon=True)
# sentiment_thread.start()

# print("✓ Producer de sentimentos iniciado (120s, 2 msgs/seg)")

In [ ]:
# OPCIONAL: Criar tópico para sentimentos
# Descomentar para executar

# !docker exec kafka kafka-topics --create \
#   --bootstrap-server localhost:9092 \
#   --topic sentiment-input \
#   --partitions 3 \
#   --replication-factor 1 \
#   --if-not-exists

# print("✓ Tópico 'sentiment-input' criado")

### 8.4.6 Executar Pipeline com Análise de Sentimentos (OPCIONAL)

**INSTRUÇÕES:**
1. Descomentar as células abaixo para executar
2. Ou executar diretamente se quiser testar análise de sentimentos
3. Criar novo tópico Kafka `sentiment-input`
4. Iniciar producer com mensagens variadas
5. Iniciar consumer com análise VADER

**NOTA:** Esta é uma extensão opcional. O pipeline principal (Seções 1-8.3) já está completo.

In [ ]:
# Índice para mensagens com análise de sentimentos
SENTIMENT_INDEX = "social-sentiment"

# Mapping otimizado
sentiment_mapping = {
    "mappings": {
        "properties": {
            "user": {"type": "keyword"},
            "text": {"type": "text"},
            "timestamp": {"type": "date"},
            "platform": {"type": "keyword"},
            "sentiment_classification": {"type": "keyword"},
            "sentiment_compound": {"type": "float"},
            "sentiment_pos": {"type": "float"},
            "sentiment_neu": {"type": "float"},
            "sentiment_neg": {"type": "float"},
            "indexed_at": {"type": "date"}
        }
    }
}

# Deletar índice existente
if es.indices.exists(index=SENTIMENT_INDEX):
    es.indices.delete(index=SENTIMENT_INDEX)
    print(f"  - Índice '{SENTIMENT_INDEX}' deletado")

# Criar índice
es.indices.create(index=SENTIMENT_INDEX, body=sentiment_mapping)
print(f"✓ Índice '{SENTIMENT_INDEX}' criado")
print("  - Campos: user, text, timestamp, platform")
print("  - Sentimentos: classification, compound, pos, neu, neg")

### 8.4.5 Criar Índice Elasticsearch para Sentimentos

In [ ]:
class SentimentElasticsearchConsumer:
    """Consumer que analisa sentimentos e indexa no Elasticsearch"""
    
    def __init__(self, bootstrap_servers, topic, es_client, es_index):
        self.consumer = KafkaConsumer(
            topic,
            bootstrap_servers=bootstrap_servers,
            value_deserializer=lambda m: json.loads(m.decode('utf-8')),
            auto_offset_reset='latest',
            enable_auto_commit=True,
            group_id='sentiment-consumer-group'
        )
        self.es = es_client
        self.index = es_index
        self.analyzer = SentimentIntensityAnalyzer()
        self.running = False
        self.document_count = 0
        self.sentiment_stats = {"positive": 0, "neutral": 0, "negative": 0}
        
    def analyze_sentiment(self, text):
        """Analisa sentimento do texto"""
        scores = self.analyzer.polarity_scores(text)
        
        # Classificar
        if scores['compound'] >= 0.05:
            classification = "positive"
        elif scores['compound'] <= -0.05:
            classification = "negative"
        else:
            classification = "neutral"
        
        return {
            "classification": classification,
            "compound": scores['compound'],
            "positive": scores['pos'],
            "neutral": scores['neu'],
            "negative": scores['neg']
        }
    
    def start(self, duration_seconds=120, batch_size=30):
        """Inicia consumer com análise de sentimentos"""
        self.running = True
        start_time = time.time()
        batch = []
        
        print(f"▶ Consumer Sentiment+ES iniciado ({duration_seconds}s)")
        
        try:
            for message in self.consumer:
                if (time.time() - start_time) > duration_seconds:
                    break
                
                data = message.value
                text = data.get("text", "")
                
                # Analisar sentimento
                sentiment = self.analyze_sentiment(text)
                
                # Preparar documento
                doc = {
                    "_index": self.index,
                    "_source": {
                        "user": data.get("user"),
                        "text": text,
                        "timestamp": data.get("timestamp"),
                        "platform": data.get("platform"),
                        "sentiment_classification": sentiment["classification"],
                        "sentiment_compound": sentiment["compound"],
                        "sentiment_pos": sentiment["positive"],
                        "sentiment_neu": sentiment["neutral"],
                        "sentiment_neg": sentiment["negative"],
                        "indexed_at": datetime.now().isoformat()
                    }
                }
                
                batch.append(doc)
                self.document_count += 1
                self.sentiment_stats[sentiment["classification"]] += 1
                
                # Indexar em lote
                if len(batch) >= batch_size:
                    success, _ = bulk(self.es, batch)
                    print(f"  [{self.document_count}] Indexados {success} docs | "
                          f"Pos: {self.sentiment_stats['positive']} | "
                          f"Neu: {self.sentiment_stats['neutral']} | "
                          f"Neg: {self.sentiment_stats['negative']}")
                    batch = []
            
            # Indexar documentos restantes
            if batch:
                success, _ = bulk(self.es, batch)
                print(f"  [Final] Indexados {success} documentos")
        
        except KeyboardInterrupt:
            print("\n⏸ Consumer interrompido")
        finally:
            self.stop()
    
    def stop(self):
        """Para consumer"""
        self.running = False
        self.consumer.close()
        print(f"■ Consumer parado. Total: {self.document_count} docs")
        print(f"  Distribuição: Positivo={self.sentiment_stats['positive']}, "
              f"Neutro={self.sentiment_stats['neutral']}, "
              f"Negativo={self.sentiment_stats['negative']}")

print("✓ Consumer com análise de sentimentos criado")

### 8.4.4 Consumer com Análise de Sentimentos + Elasticsearch

In [ ]:
# Dataset expandido com mensagens de sentimento variado
SENTIMENT_MESSAGES = [
    # Positivas
    "I absolutely love working with Spark Streaming! It's amazing and powerful!",
    "Kafka integration is fantastic and makes everything so much easier",
    "This distributed system is incredibly efficient and reliable",
    "Great tools for big data processing, highly recommended!",
    "Excellent performance with Elasticsearch indexing, very impressed",
    
    # Neutras
    "Apache Spark processes data using structured streaming",
    "Kafka is a distributed message broker for event streaming",
    "Elasticsearch provides indexing and search capabilities",
    "The pipeline consists of producer consumer and streaming components",
    "Docker containers run the infrastructure services",
    
    # Negativas
    "Configuration is frustrating and takes too much time",
    "Terrible documentation, very difficult to understand",
    "Performance is poor and disappointing with large datasets",
    "This setup is awful and causes many problems",
    "I hate dealing with version compatibility issues",
    
    # Mistas
    "Spark is powerful but the learning curve is steep and challenging",
    "Good results but the process is slow and requires patience",
    "Effective solution despite some annoying configuration issues"
]

class SentimentProducer(SocialMediaProducer):
    """Producer que envia mensagens com sentimentos variados"""
    
    def generate_message(self):
        """Gera mensagem do dataset de sentimentos"""
        return {
            "user": random.choice(SAMPLE_USERS),
            "text": random.choice(SENTIMENT_MESSAGES),
            "timestamp": datetime.now().isoformat(),
            "platform": "SimulatedSocial",
            "message_id": self.message_count
        }

print("✓ Producer com mensagens de sentimento variado criado")
print(f"  - Total de templates: {len(SENTIMENT_MESSAGES)}")
print(f"  - Positivas: ~6 | Neutras: ~5 | Negativas: ~5 | Mistas: ~3")

### 8.4.3 Producer com Mensagens Variadas (Sentimento Misto)

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Inicializar analisador
analyzer = SentimentIntensityAnalyzer()

# Exemplos de teste
test_messages = [
    "Apache Spark is amazing for big data processing!",
    "Learning distributed systems is challenging but rewarding",
    "This pipeline is terrible and doesn't work at all",
    "Kafka provides reliable message streaming capabilities",
    "I hate dealing with configuration files"
]

print("=== Teste de Análise de Sentimentos ===\n")

for msg in test_messages:
    scores = analyzer.polarity_scores(msg)
    
    # Classificar baseado no score composto
    if scores['compound'] >= 0.05:
        sentiment = "POSITIVO 😊"
    elif scores['compound'] <= -0.05:
        sentiment = "NEGATIVO 😞"
    else:
        sentiment = "NEUTRO 😐"
    
    print(f"Mensagem: {msg[:60]}...")
    print(f"  Compound: {scores['compound']:.3f}")
    print(f"  Positivo: {scores['pos']:.3f} | Neutro: {scores['neu']:.3f} | Negativo: {scores['neg']:.3f}")
    print(f"  → Classificação: {sentiment}\n")

### 8.4.2 Teste de Análise de Sentimentos

In [ ]:
!pip install -q vaderSentiment

print("✓ VADER Sentiment instalado")
print("  - Biblioteca: vaderSentiment")
print("  - Modelo: Léxico pré-treinado para redes sociais")

### 8.4.1 Instalação VADER

---

## 8.4 Extensão Opcional: Análise de Sentimentos (ML)

**NOTA:** Esta seção é opcional e demonstra como enriquecer o pipeline com análise de sentimentos usando Machine Learning.

### Por que Análise de Sentimentos?

Análise de sentimentos permite:
- Classificar mensagens como positivas, neutras ou negativas
- Identificar tendências de sentimento em tempo real
- Gerar alertas para sentimentos extremos
- Complementar análise de word count com contexto emocional

### Biblioteca Utilizada: VADER

**VADER (Valence Aware Dictionary and sEntiment Reasoner)**

- Desenvolvido especificamente para textos de redes sociais
- Não requer treinamento (modelo léxico pré-construído)
- Considera contexto (negação, intensificadores, pontuação)
- Retorna scores: positivo, negativo, neutro e composto

**Referência Principal:**
> Hutto, C.J. & Gilbert, E.E. (2014). VADER: A Parsimonious Rule-based Model for Sentiment Analysis of Social Media Text. Eighth International Conference on Weblogs and Social Media (ICWSM-14). Ann Arbor, MI, June 2014.
> 
> [Paper Original](http://comp.social.gatech.edu/papers/icwsm14.vader.hutto.pdf)

**Diferencial deste Trabalho:**
- Integração com pipeline Spark Streaming (não batch)
- Indexação simultânea de word count + sentimentos no Elasticsearch
- Visualização de sentimentos no Kibana em tempo real
- Implementação autocontida no notebook

---

## 9. Conclusões e Observações

### 9.1 Objetivos Alcançados

✅ **Entrada via Kafka:**
- Producer simulando mensagens de rede social implementado
- Justificativa documentada para não uso de Discord (OAuth, conexão persistente, reprodutibilidade)
- Alternativa baseada em geração sintética de dados com características realistas

✅ **Pipeline Spark Structured Streaming:**
- Leitura de stream do Kafka com deserialização JSON
- Processamento de word count com janelas temporais (30s/10s slide)
- Watermark configurado para lidar com eventos atrasados
- Publicação de resultados em tópico Kafka de saída

✅ **Integração com Elasticsearch:**
- Índice criado com mapping otimizado
- Consumer Kafka indexando resultados em tempo real
- Bulk indexing para performance

✅ **Visualização no Kibana:**
- Instruções detalhadas para criação de Tag Cloud
- Alternativas documentadas (Bar Chart, Data Table, Treemap)
- Dashboard com múltiplas visualizações sugeridas

✅ **Execução Autocontida:**
- **100% das operações executadas em células do notebook**
- Nenhuma dependência de scripts externos
- Infraestrutura gerenciada via docker-compose

### 9.2 Arquitetura Implementada

```
[Producer Python]  →  [Kafka: social-input]  →  [Spark Streaming]  →  [Kafka: wordcount-output]
                                                        ↓
                                                  [Console Debug]
                                                        
[ES Consumer]  ←  [Kafka: wordcount-output]
      ↓
[Elasticsearch: wordcount-realtime index]
      ↓
[Kibana Dashboard: Tag Cloud + Metrics]
```

### 9.3 Parâmetros de Configuração

| Componente | Parâmetro | Valor |
|------------|-----------|-------|
| Producer | Taxa de mensagens | 3 msgs/seg |
| Producer | Duração | 180 segundos |
| Spark | Janela temporal | 30 segundos |
| Spark | Slide | 10 segundos |
| Spark | Watermark | 1 minuto |
| Kafka | Partições (input) | 3 |
| Kafka | Partições (output) | 3 |
| ES Consumer | Batch size | 30 documentos |

### 9.4 Melhorias Possíveis

- **Análise de Sentimentos:** Integrar modelo VADER ou TextBlob para classificar mensagens
- **Filtro de Stop Words:** Remover palavras comuns (the, and, is) para resultados mais relevantes
- **Agregações Múltiplas:** Word count por usuário, por período do dia, etc.
- **Alertas:** Configurar alertas no Kibana para palavras específicas
- **Persistência:** Kafka com replicação para tolerância a falhas

### 9.5 Referências

- [Apache Spark Structured Streaming Guide](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html)
- [Kafka Documentation](https://kafka.apache.org/documentation/)
- [Elasticsearch Python Client](https://elasticsearch-py.readthedocs.io/)
- [Kibana Visualizations](https://www.elastic.co/guide/en/kibana/current/dashboard.html)

---

**Notebook criado por:** Edilberto Cantuaria  
**Data:** 29 de Novembro de 2025  
**Disciplina:** PSPD - UnB